# Import libraries

In [1]:
import pandas as pd
import uuid
import os 
from collections import defaultdict
from datetime import date
import datetime
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

### Get current dir and move to appropriate location to import file

In [2]:
#Move directories to find file
os.getcwd()
os.chdir('../data')
print(os.path.abspath(os.curdir))

/Users/andreagarcia/Documents/Data Science Grad Program/Fall 2021/Capstone/Development/data


### Import file to obtain PIDs 

In [3]:
file_path = "all_programs_original.xlsx"
full_file = pd.read_excel (file_path)

In [85]:
full_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615165 entries, 0 to 615164
Data columns (total 40 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   EMPLID             615165 non-null  int64  
 1   TERM_ID            615165 non-null  int64  
 2   ADMIT_TERM         615165 non-null  object 
 3   ADMIT_STRM         615165 non-null  int64  
 4   ACAD_YR            615165 non-null  object 
 5   TOT_ACT            121526 non-null  float64
 6   TOT_SAT            189026 non-null  float64
 7   TOT_SAT_W          187130 non-null  float64
 8   DATE_OF_BIRTH      615165 non-null  int64  
 9   ETHNICITY          615165 non-null  object 
 10  STUDENT_GENDER     615165 non-null  object 
 11  ADMIT_COLLEGE      615165 non-null  object 
 12  ADMIT_DEPARTMENT   615165 non-null  object 
 13  FL_RESY            615165 non-null  object 
 14  emplid2            615165 non-null  int64  
 15  ACAD_YR3           615165 non-null  object 
 16  TE

In [7]:
id_list = full_file['EMPLID'].unique().tolist()

In [9]:
id_df = pd.DataFrame(id_list, columns=['EMPLID'], dtype=int)

### Export IDs File

In [27]:
id_df.to_excel('student_ids.xlsx', encoding='utf-8')

# Import student ids file

In [4]:
print(os.getcwd())

/Users/andreagarcia/Documents/Data Science Grad Program/Fall 2021/Capstone/Development/data


In [5]:
path = "student_ids.xlsx"
anon = pd.read_excel (path)

In [6]:
anon.drop('Unnamed: 0', axis=1, inplace=True)

In [7]:
anon.reset_index(drop=True, inplace=True) 

### Create uuid

In [11]:
anon['uuid'] = [uuid.uuid4() for _ in range(len(anon.index))]

In [ ]:
anon

In [18]:
anon['uuid'].nunique()

38738

### Export mapping file

In [53]:
anon.to_excel('student_ids_mapping.xlsx', encoding='utf-8')

## Import graduated students file

In [9]:
deg_path = "degrees_awarded_original.xlsx"
deg_file = pd.read_excel (deg_path)

In [11]:
def convertStrm(term_id):
    term = str(term_id)
    
    strm = "1" + term[2:4] + term[-1:]
    return int(strm)

In [12]:
deg_file['grad_strm'] = deg_file['Term_id'].apply(convertStrm) 

# Merge degree file with main student file

In [15]:
deg_data = pd.merge(full_file, deg_file, how='left', on=['EMPLID'],
         suffixes=('_og', '_deg'), copy=True, indicator=False,
         validate=None)

In [16]:
deg_data['EMPLID'].nunique()

38738

In [ ]:
deg_data.sample(5)

In [18]:
deg_data['grad_strm'] = deg_data['grad_strm'].fillna(-1)


In [19]:
def compareStrm(admit, grad): 
    admit = int(admit)
    grad = int(grad)

    return '' if admit > grad else grad

In [20]:
deg_data['grad_strm_new'] = deg_data.apply(lambda x: compareStrm(x['ADMIT_STRM'], x['grad_strm']), axis=1)

In [ ]:
deg_data.info()

In [23]:
def test(admit, grad): 
    admit = int(admit)
    grad = int(grad)
    return True if admit > grad else False

In [24]:
deg_data['test_flag'] = deg_data.apply(lambda x: test(x['ADMIT_STRM'], x['grad_strm']), axis=1)

In [25]:
test_df = deg_data.groupby(['test_flag']).agg({'EMPLID': ['nunique']})


In [113]:
deg_data.to_excel('deg_programs_all.xlsx', encoding='utf-8')

# Merge mapping file with main student file

In [28]:
map_path = "student_ids_mapping.xlsx"
map_file = pd.read_excel (map_path)

In [29]:
anon_data = pd.merge(deg_data, map_file, how='left', on=['EMPLID'],
         suffixes=('_og', '_anon'), copy=True, indicator=False,
         validate=None)

# Calculate Age

In [65]:
# Use to calculate the age of the student
'''
dob: student's date of birth
term: the term used to calculate the date to compare to the student's date of birth (admission, enrollment, etc.)
      this should be the four digit term code and not the six digit term id
'''
def ageCalculator(dob, term):
    # formatting date of birth
    dob = str(dob)
    dateOfBirth = date(year=int(dob[0:4]), month=int(dob[4:6]), day=int(dob[6:8]))
    
    # preparing to format pseudo date for term
    season = int(str(term)[-1])
    
    if len(str(term)) == 4:
        year = int('20' + str(term)[1:3])
    elif len(str(term)) == 6:
        year = int(str(term)[0:4])
    
    term_date = datetime.date(year, season, 1)
        
    try:
        birthday = dateOfBirth.replace(year = term_date.year)
 
    # used to deal with 2/29 birthdays (leap years)
    except ValueError:
        birthday = dateOfBirth.replace(year = term_date.year,
                  month = dateOfBirth.month + 1, day = 1)
 
    if birthday > term_date:
        return term_date.year - dateOfBirth.year - 1
    else:
        return term_date.year - dateOfBirth.year
    

In [66]:
# Calculate age at time of admission
anon_data['age_admission'] = anon_data.apply(lambda x: ageCalculator(x['DATE_OF_BIRTH'], x['ADMIT_STRM']), axis=1)

In [67]:
# Calculate age at time of enrollment term
anon_data['age_enrlterm'] = anon_data.apply(lambda x: ageCalculator(x['DATE_OF_BIRTH'], x['TERM_ID4']), axis=1)

In [75]:
anon_data.shape

(637327, 55)

In [33]:
# Keep only grades with a grade point value
anon_data['CRS_GRADE']=anon_data['CRS_GRADE'].apply(lambda x: x.strip())
grades_to_keep = ['A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'D-', 'F', 'P']
anon_data_grades = anon_data.loc[anon_data['CRS_GRADE'].isin(grades_to_keep)].copy()

In [34]:
anon_data_grades.shape

(564787, 53)

In [ ]:
anon_data_grades.loc[anon_data_grades['CRS_GRADE'] == 'F']

# Delete PII (DOB and EMPLID)

In [36]:
cols_to_drop = [
    'TERM_ID6',
    'Gender',
    'Ethnicity',
    'emplid2',
    'EMPLID5',
    'test_flag',
    'EMPLID',
    'DATE_OF_BIRTH',
    'Unnamed: 0']

In [37]:
anon_data_grades.drop(cols_to_drop, axis=1, inplace=True)

# Export Anonymized file

In [39]:
anon_data_grades.to_excel('anon_data_all.xlsx', encoding='utf-8')